In [9]:
#請將以下程式碼產生的資料,儲存為csv檔和xlsx檔

import requests
from requests import Response
from pydantic import BaseModel,RootModel,Field,field_validator,field_serializer
from datetime import datetime
from pprint import pprint
import numpy as np
import pandas as pd

url="https://data.ntpc.gov.tw/api/datasets/c51d5111-c300-44c9-b4f1-4b28b9929ca2/json?size=1000"
try:
    respons:Response=requests.request("GET",url)
    respons.raise_for_status()
except Exception:
    print("下載錯誤")
else:
    content=respons.text

class Factorydata(BaseModel):
    名稱:str=Field(alias="organizer")
    註冊編號:str=Field(alias="no")
    地址:str=Field(alias="address")
    統編:str=Field(alias="tax_id_number")
    緯度:float|None=Field(alias="wgs84ax")
    經度:float|None=Field(alias="wgs84ay")
    日期:datetime=Field(alias="date")

    @field_validator("緯度","經度",mode="before")
    @classmethod
    def slice_str(cls,value:str)->float|None:
        if value==None:
            return None
        else:
            return round(float(value),ndigits=5)

    @field_serializer("日期")
    def date_serializer(self,date:datetime)->str:
        return date.strftime(f"中華民國{date.year-1911}年%m月%d日")
    
class Factory(RootModel):
    root:list[Factorydata]

factory:Factory=Factory.model_validate_json(content)
factory_list=factory.model_dump()

df1 = pd.DataFrame(factory,
                   columns=['名稱','地址','日期','統編','經度','緯度','註冊編號'],
                   index=0)
df1

ValueError: 7 columns passed, passed data had 2 columns